In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import xarray as xr
import numpy as np
from glob import glob
from yllib.function import mysom
from yllib.function import myfunc
from yllib.plot import plot_map
from cartopy import crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl
%config InlineBackend.figure_format='retina'
%matplotlib inline

In [2]:
year1   = 1981
year2   = 2020
cold_months  = [10,11,12,1,2,3]
warm_months  = [4,5,6,7,8,9]
mapsize = (4,4)
factor  = ('z')
map_extent = [160,-20+360,25,90]
disp_order_cold = [1,0,3,2]
disp_order_warm = [3,1,2,0]

In [3]:
bmu_cold = xr.open_dataset(f"som/kmean_cold_z_{mapsize[0]}x{mapsize[1]}.nc")['bmu']
bmu_warm = xr.open_dataset(f"som/kmean_warm_z_{mapsize[0]}x{mapsize[1]}.nc")['bmu']

In [4]:
data = xr.open_dataset('data/hgt_daily_anomaly_globe.nc')
# select season
data_cold = data.sel(time=data['time.month'].isin(cold_months))['z500']
data_warm = data.sel(time=data['time.month'].isin(warm_months))['z500']

In [5]:
def calc_transition(bmu, data, season):
    bmu.coords["time"] = data.coords["time"]

    order = disp_order_cold if season=='cold' else disp_order_warm
    from itertools import groupby
    pers = []
    print(bmu.size)
    for ibmu in order:
        a = bmu.where(bmu==ibmu, drop=True)
        print(a.size)
        _time = a.time + pd.Timedelta("1D")
        _time = np.intersect1d(_time, bmu.time)
        _bmu  = bmu.sel(time=_time)
        _bmu  = _bmu.where(bmu!=ibmu, drop=True)
        count = [_bmu.where(_bmu==i, drop=True).size/a.size for i in np.arange(4)]
        count[ibmu] = 1 - np.sum(count)
        print([int(count[i]*a.size) for i in order])
        print([f'{count[i]*100:.1f}' for i in order])

        print(_bmu.size)
        # count.to_netcdf(f'som/som_persistence_{season}_{ibmu}.nc')
    return pers

### Calculate persistence

In [6]:
pers_cold = calc_transition(bmu_cold, data_cold, 'cold')


7290
1552
[1274, 167, 27, 83]
['82.1', '10.8', '1.8', '5.3']
278
2690
[143, 2218, 196, 132]
['5.3', '82.5', '7.3', '4.9']
472
1543
[64, 181, 1252, 46]
['4.1', '11.7', '81.1', '3.0']
291
1505
[72, 122, 65, 1246]
['4.8', '8.1', '4.3', '82.8']
259


In [7]:
# pers_cold = calc_transition(bmu_cold, data_cold, 'cold')
pers_warm = calc_transition(bmu_warm, data_warm, 'warm')

7320
1364
[1019, 174, 53, 118]
['74.7', '12.8', '3.9', '8.7']
345
2631
[151, 1909, 291, 280]
['5.7', '72.6', '11.1', '10.6']
722
1465
[39, 307, 1077, 42]
['2.7', '21.0', '73.5', '2.9']
388
1860
[155, 239, 47, 1419]
['8.3', '12.8', '2.5', '76.3']
441
